# 00 The basics of using Spark and Jupyter notebook


## Definitions
### Application
A user program built on Spark using its APIs. It consists of a driver program and executors on the cluster.

### SparkSession
An object that provides a point of entry to interact with underlying Spark functionality and allows programming Spark with its APIs. In an interactive Spark shell, the Spark driver instantiates a SparkSession for you, while in a Spark application, you create a SparkSession object yourself.

### Job
A parallel computation consisting of multiple tasks that gets spawned in response to a Spark action (e.g., save(), collect()).

### Stage
Each job gets divided into smaller sets of tasks called stages that *depend* on each other.
This means that a stage is executed serially

### Task
A single unit of work or execution that will be sent to a Spark executor. Tasks can be executed in parallel

<hr>

Depending on the configuration, when starting the notebook that is connected to Spark, a SparkSession and SparkContext are already created.

In [ ]:
if 'spark' in dir():
    print("spark context is already created for you!")
else: print("You need to create your own SparkSession object")

In any case, we can ask for a spark session, and we will get the existing or a new one, *maybe* with the configuration modification we specify.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import*

In [ ]:
# Before getting/creating the Session, we can try to modify parameters. 
spark = SparkSession.builder.appName('00 the basics')\
    .getOrCreate()
sc = spark.sparkContext
# keep only important logs
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# ONLY when running in jupyter:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [ ]:
# see what version of Spark we are running.
spark

You should get something like
```
SparkSession - in-memory

SparkContext

Spark UI

Version           v3.2.0  << should be at least 3.2.0
Master            local[*] << local means Spark is running on one machine, '*' means it uses all the cores in this machine
AppName           00 the basics
```

The Spark UI is available once the session object is created

Now open this link to see the Spark UI: 
http://localhost:4040

# What is Spark?
Apache Spark is an open-source cluster computing framework.

Built on top of Hadoop MapReduce.

Utilizes In-memory computing.

Originally developed at UC Berkeley (2009).

# Basic Dataframe operations

## The RDD  - Resilient Distributed Datasets
- Spark's primary data abstraction.
- A fault-tolerant collection of elements (any type), partitioned across the nodes of the cluster
and capable of accepting parallel operations.
- Sharing data across multiple stages of an iterative computation.<br>
Efficency is accomplished in two ways:
  - Ensures that the partitions that are assigned to each worker node are maintained between iterations to avoid shuffling data.
  - Avoids writing and reading from HDFS in between iteration jobs by keeping the RDDs in memory, since the assignment to workers is maintained from one iteration to the next, this is feasible.
- Immutable.
- RDD Operations: **Transformations & Actions**

### Transformations

Transformations are operations that will not be completed at the time
you write and execute the code in a cell - they will only get executed
once you have called a action. An example of a transformation might be
to convert an integer into a float or to filter a set of values.

### Actions

Actions are commands that are computed by Spark right at the time of
their execution. They consist of running all of the previous
transformations in order to get back an actual result. An action is
composed of one or more jobs which consists of tasks that will be
executed by the workers in parallel where possible.


Here are some simple examples of transformations and actions. Remember,
these are not all the transformations and actions - this is just a short
sample of them. We'll get to why Apache Spark is designed this way
shortly!

| Transformations(*lazy*) | Actions |
|-------------------------|---------|
| select                  | show    |
| distinct                | count   |
| groupBy                 | collect |
| sum                     | save    |
| orderBy                 |         |
| filter                  |         |
| limit                   |         |


## Directed Acyclic Graph (DAG)
* Vertices are RDD, edges are Transformations

* Generalization of MapReduce

* Action divides DAG to Stages

* This model lets Spark decide which calculations should be recomputed and which can be reused (shown as "skipped" in the user interface)

<img src="https://i.stack.imgur.com/yQf7L.png" >


## Spark​ Structured​ API
The Structured APIs are a tool for manipulating all sorts of data, from unstructured log files to semi-structured CSV files and highly structured Parquet files. These APIs refer to three core types of distributed collection APIs:

* Datasets (Java and Scala API only).
* DataFrames.
* SQL tables and views


## The Dataframe

A DataFrame is the most common Structured API and simply represents a table of data with
rows and columns. The list that defines the columns and the types within those columns is called
the schema. You can think of a DataFrame as a spreadsheet with named columns.

Each column has a name and a type (e.g. StringType) .

Like RDD, the DataFrame is 
Immutable, in-memory, resilient, distributed collection of data.

It allows better optimizations (memory management and optimized execution plan), and was added to spark in version 2.


see [Data types](https://spark.apache.org/docs/latest/sql-ref-datatypes.html)


### DataFrame vs. RDD: which is better?
Usually, DataFrame. Consider RDD if:

* Unstructured data (text, media).
* Specific execution control needed.
* Data manipulation with functional programming concepts.

In practice Spark DF rides on RDD

## Working with RDD

In [ ]:
# parallelize() will copy the python object to the JVM, then cut it into partitions according to some rule, 
# and then send the partitions to the worker nodes for processing.
nums = sc.parallelize([1, 2, 3, 4, 555])
print('Type:', type(nums))
print('Count:', nums.count())

# each node runs the map() on the partitions it has.
# The collect() collects all the results (partitions of the result RDD) to the driver node,
# Then copies the data from the JVM to the python process.
print('Squared:', nums.map(lambda x: x**2).collect())

In [ ]:
import os
print(f"SparkContext default Number of partitions: {sc.defaultParallelism}")
print(f"Number of CPUs in the system: {os.cpu_count()}")
print(f"Number of partitions in nums RDD: {nums.getNumPartitions()}")

In [ ]:

# Generate a random double RDD that contains 1 million i.i.d. values drawn from the
# standard normal distribution `N(0, 1)`, evenly distributed in 10 partitions.
u = RandomRDDs.normalRDD(spark, 1000000, 10)
u = u.map(lambda x: (x,)) # convert to tuple so we can transorm into DF

## Working with Dataframe

The RDD is the basic building block, and usually we will want to use a higher level object: The Dataframe wraps the RDD and exposes a convenient API.

DataFrame always has a schema

### The Schema
 A schema is a StructType made up of a number of fields, StructFields, that have a name, type, and a Boolean flag which specifies whether that column can contain missing or null values.
 
When reading data from a file, the schema can be inferred automatically at a cost of reading the data more than once.

### The Row

In Spark, each row in a DataFrame is a single record. Spark represents
this record as an object of type Row. Spark manipulates Row objects
using column expressions in order to produce usable values. Row objects
internally represent arrays of bytes. The byte array interface is never
shown to users because we only use column expressions to manipulate
them.



In [ ]:
schema = StructType([  StructField('c1', FloatType(), False)])
# we can move from RDD to Dataframe and back. 
df = spark.createDataFrame(u, schema) #something is wrong here with the schema

# each DF has a schema:
df.printSchema()
df.show(5)

In [ ]:
#Tip: To see the table in nicer format, convert it to Pandas:
df.limit(5).toPandas()

In [ ]:
# get the RDD from the Dataframe
r = df.rdd
type(r)

In [ ]:
# Create a simple dataframe
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)

df = rdd.toDF()
df.printSchema()
df.show(truncate=False)

In [ ]:
#  Transformation:
# create an array of M numbers
# This is fast since it is a TRANSFORMATION. 
# It is just an execution plan, so if allocating M numbers
# will use all the memory on this machine, we will not see it now.
M = 100*1000 *1000
myRange = spark.range(M).toDF("number")
nums_doubled_df = myRange.selectExpr("(number * 2) as value")

In [ ]:
# Actions:
# Collect the dataframe from all worker nodes (the executors) to the driver program.
# if this is too large a "Java heap space exception" will happen, and then you have to restart your kernel.
# Since we use Pyspark, this data is then *copied* from the JVM to the python runtime.

the_big_list = myRange.collect()
print(nums_doubled_df.take(5))

In [ ]:
type(the_big_list), len(the_big_list)

In [ ]:
divisBy2 = myRange.where("number % 2 = 0")
print("Count: ",divisBy2.count())
divisBy2.sort('number').show(8)

## Example: text processing using RDD

In [ ]:
toxic_rdd = sc.textFile('../data/toxic.txt')
print('Type:', type(toxic_rdd))
print('Count (rows):', toxic_rdd.count())

<hr>

In [ ]:
# Split rows to words:
toxic_words = toxic_rdd.flatMap(lambda row: row.split())

In [ ]:
# Top 10 most frequent words:
toxic_words.map(lambda word: (word.casefold(), 1)) \
        .reduceByKey(lambda a, b: a + b) \
            .sortBy(lambda t: t[1], ascending=False) \
                .take(10)

In [ ]:
# counting 'baby':
toxic_words.filter(lambda word: word.lower() == 'baby').count()

# Check yourself

Try to increase M (in the range() above ) by 1000, and run the code again. What do you expect? 